In [27]:
import pandas as pd
import joblib

In [28]:

# Function to classify customers based on predicted income
def predict_income(df, model):
    # Make predictions
    predicted_income = list(model.predict(df))
    
    # Classify customers based on predicted income
    income_data = []
    for i in predicted_income:
        if i <= 2500:
            dic = {'predicted_income': i, 'credit_limit': 3000, 'card_class': 'standard', 'loan_granted': 18000}
        elif i <= 5000:
            dic = {'predicted_income': i, 'credit_limit': 6000, 'card_class': 'gold', 'loan_granted': 36000}
        elif i <= 7500:
            dic = {'predicted_income': i, 'credit_limit': 8500, 'card_class': 'platinum', 'loan_granted': 54000}
        elif i <= 10000:
            dic = {'predicted_income': i, 'credit_limit': 15000, 'card_class': 'black', 'loan_granted': 72000}
        elif i <= 15000:
            dic = {'predicted_income': i, 'credit_limit': 20000, 'card_class': 'black', 'loan_granted': 100000}
        else:
            dic = {'predicted_income': i, 'credit_limit': 'unlimited', 'card_class': 'black', 'loan_granted': 150000}
        income_data.append(dic)
    
    # Convert to DataFrame
    income_data = pd.DataFrame(income_data)
    
    # Add classification results to the original DataFrame
    df = df.reset_index(drop=True)
    income_data = income_data.reset_index(drop=True)
    df = pd.concat([df, income_data], axis=1)
    
    return df

In [29]:
# Load new data
novos_dados = pd.read_csv('./input/previsao_de_renda.csv')

In [30]:
# Load the trained model
model_path = 'prevision_income_model.pkl'
tree = joblib.load(model_path)

In [31]:
# Apply the same transformations as in the training phase
novos_dados['sexo'] = novos_dados['sexo'].map({'M': 1, 'F': 0})
novos_dados_dummies = pd.get_dummies(novos_dados, dtype=int)
novos_dados_dummies.columns = [col.lower().replace(' ', '_').replace('á', 'a').replace('ã', 'a').replace('ú', 'u').replace('ó', 'o') 
                               for col in novos_dados_dummies.columns]

In [32]:
# Select significant variables (same as in training)
var_significant = [
    'sexo', 
    'qtd_filhos', 
    'tipo_renda_assalariado',
    'tipo_renda_servidor_publico',
    'educacao_secundario',
    'educacao_superior_incompleto',
    'estado_civil_casado',
    'estado_civil_solteiro',
    'idade',
    'tempo_emprego',
    'qt_pessoas_residencia'
]
novos_dados_significant = novos_dados_dummies[var_significant]

In [33]:
# Make predictions and classify customers
resultados = predict_income(novos_dados_significant, tree)

In [34]:
# Save the results to a CSV file
resultados.to_csv('./output/prevision_income_results.csv', index=False)

In [35]:
# Display the first few rows of the results
print(resultados.head())

   sexo  qtd_filhos  tipo_renda_assalariado  tipo_renda_servidor_publico  \
0     0           0                       0                            0   
1     1           0                       1                            0   
2     0           0                       0                            0   
3     0           1                       0                            1   
4     1           0                       1                            0   

   educacao_secundario  educacao_superior_incompleto  estado_civil_casado  \
0                    1                             0                    0   
1                    0                             0                    1   
2                    0                             0                    1   
3                    0                             0                    1   
4                    1                             0                    0   

   estado_civil_solteiro  idade  tempo_emprego  qt_pessoas_residencia  \
0      